In [1]:
import dotenv
import os
import re
import os
import glob
import pandas as pd
import numpy as np
import openai
import json
from pathlib import Path
import tiktoken

dotenv.load_dotenv()

OPENAI_KEY = os.environ.get("OPENAI_API_KEY")

q_and_a_behavior_df = pd.read_csv('behavior_level.csv')
q_and_a_experiment_df = pd.read_csv('experimental_level.csv')

master_db_df = pd.read_csv('MasterDB_Full.csv')
master_db_df['Paper_Exp_ID_better'] = master_db_df.Paper_Exp_ID.str.extract(r'([A-Z]\_\d+)')

In [2]:
article_ids_list = [
    "A_114",
    "A_116",
    "A_137",
    "A_152",
    "A_168",
    "A_168",
    "A_168",
    "A_181",
    "A_197",
    "A_19",
    "A_228",
    "A_30",
    "A_31",
    "A_39",
    "A_39",
    "A_55",
    "A_62",
    "A_76",
    "A_87",
]

file_dict = {}

current_directory = os.getcwd()

relative_path = "../server/files/sample_paper"

folder_path = os.path.join(current_directory, relative_path)

markdown_pattern = "*.mmd"
markdown_files = glob.glob(os.path.join(folder_path, markdown_pattern))

print(markdown_files)

pattern_id = r"([A-Z]\_\d+)"

for markdown_file in markdown_files:
    match = re.search(pattern_id, markdown_file)
    if match:
        code = match.group(1)
        if code in file_dict:
            file_dict[code].append(markdown_file)
        else:
            file_dict[code] = [markdown_file]
    else:
        print("No match found.")

['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_55_2022_TryingToMake.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_116_2020_AdvanceCounterBiasing.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_76_2021_EyeCuesIncrease.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_228a_2022_InvestmentInIdentity.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_114b_2021_WhenNudgesArent.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_31_2022_ImprovingTaxpayerRegistration.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_137a_2020_NudgingGenerosityIn.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_39b_2022_NudgingDebtorsTo.mmd', '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_62a_2021_ASoc

In [3]:
def read_files_to_string(docs: list):
    text = ""
    for d in docs:
        text += Path(d).read_text()
    return text

In [4]:
file_dict

{'A_55': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_55_2022_TryingToMake.mmd'],
 'A_116': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_116_2020_AdvanceCounterBiasing.mmd'],
 'A_76': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_76_2021_EyeCuesIncrease.mmd'],
 'A_228': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_228a_2022_InvestmentInIdentity.mmd',
  '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_228b_2022_InvestmentInIdentity.mmd'],
 'A_114': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_114b_2021_WhenNudgesArent.mmd',
  '/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_114a_2021_WhenNudgesArent.mmd'],
 'A_31': ['/Users/amir/Documents/projects/atlas/notebooks/../server/files/sample_paper/A_31_2022_ImprovingTaxpayerRegistration.mmd'],
 'A_137': ['/Users/

In [5]:
files_to_read = [
    "/Users/amir/Documents/projects/atlas/server/files/sample_paper/A_114a_2021_WhenNudgesArent.mmd",
    "/Users/amir/Documents/projects/atlas/server/files/sample_paper/A_114b_2021_WhenNudgesArent.mmd"
]

read_files_to_string(
    files_to_read
)

encoding = tiktoken.encoding_for_model("gpt-4")

encoded = encoding.encode(read_files_to_string(files_to_read))

len(encoded)

15967

## OpenAI assistant API tests

In [13]:
client = openai.OpenAI()

def get_assistant(client, assistant_name):
    my_assistants = client.beta.assistants.list(
        order="desc",
        limit="20",
    )

    assistant_dict = {"id": "", "name": "", "assistant": None}

    for assis in my_assistants.data:
        if assis.name == assistant_name:
            print(assis.id)
            print(assis.name)
            assistant_dict["id"] = assis.id
            assistant_dict["name"] = assis.name
            assistant_dict["assistant"] = client.beta.assistants.retrieve(assis.id)

    return assistant_dict

In [14]:
assistant = get_assistant(client, "Atlas")

assistant['assistant']

asst_Q7HIiIPCC2inl0NKibrVaMYR
Atlas


Assistant(id='asst_Q7HIiIPCC2inl0NKibrVaMYR', created_at=1699377999, description=None, file_ids=['file-froCPBezllc8nNauFSW3mqoT'], instructions='You are an assistant that reads through scientific articles and gives me the behavioral outcome or experiment conditions in the article and also a description for them.', metadata={}, model='gpt-4-1106-preview', name='Atlas', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [111]:
condition_prompt = {
    "text": (
        """Use the above article to answer the question at the end according to the format of the answer provided. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
        You named one of the experimental conditions in this article {condition}. 
            
        Can you answer the following question about this condition? 
        Question: {question}
        """
    )
}

article_text = read_files_to_string(files_to_read)

condition_prompt["text"].format(
    condition="Experimental Condition",
    question="What is the significance of this condition?",
    # answer_format='{"response": "true/false/unknown", "explanation": "explanation of the answer"}',
)

"Use the above article to answer the question at the end according to the format of the answer provided. \n        If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. \n        You named one of the experimental conditions in this article Experimental Condition. \n            \n        Can you answer the following question about this condition? \n        Question: What is the significance of this condition?\n        "

In [17]:
def get_condition_behavior_human_answers(master_db, article_id):
    condition_keyword_list = [
        "condition_financial",
        "condition_educational",
        "condition_forbid",
        "condition_channel_snailmail",
        "condition_channel_email",
        "condition_channel_paperlive",
        "condition_channel_inperson",
        "condition_channel_mobileapp",
        "condition_channel_online",
        "condition_channel_sms",
        "condition_channel_phone",
        "condition_channel_nothing",
    ]

    behavior_keywords_list = [
        "behavior_priority",
        "behavior_focal",
        "behavior_focal_estimate",
        "behavior_metric",
        "behavior_selfreport",
        "financial_reqt",
        "travel_reqt",
        "planning_reqt",
        "location_reqt",
        "timing_reqt",
        "prosociality",
        "delay",
        "authority",
    ]

    to_process = master_db[master_db["Paper_Exp_ID_better"] == article_id]

    first_row_entries_behavior = to_process.groupby("behavior_description").first()[
        behavior_keywords_list
    ]

    first_row_entries_condition = to_process.groupby("condition_name").first()[
        condition_keyword_list
    ]

    return first_row_entries_behavior, first_row_entries_condition


behavior_human, condition_human = get_condition_behavior_human_answers(
    master_db_df, 'A_114'
)

condition_human

,condition_financial,condition_educational,condition_forbid,condition_channel_snailmail,condition_channel_email,condition_channel_paperlive,condition_channel_inperson,condition_channel_mobileapp,condition_channel_online,condition_channel_sms,condition_channel_phone,condition_channel_nothing
condition_name,,,,,,,,,,,,
control,Y,N,N,Y,N,N,N,N,N,N,N,N
four-week card offer,Y,N,N,Y,N,N,N,N,N,N,N,N
implementation intentions (without extra lines),Y,N,N,Y,N,N,N,N,N,N,N,N
implementation intentions (without extra linse),Y,N,N,Y,N,N,N,N,N,N,N,N
negative social norm,Y,N,N,Y,N,N,N,N,N,N,N,N
positive social norm,Y,N,N,Y,N,N,N,N,N,N,N,N
social norms,Y,N,N,Y,N,N,N,N,N,N,N,N


Create a new thread 

In [18]:
prompt = condition_prompt["text"].format(
    context=article_text,
    condition="four-week card offer",
    question=(
        f"Does the condition involve financial incentives? "
        f"These need not be behaviorally informed. Note this is not about the context being financial, but the condition involving financial incentives."
        f"Note: If all participants are paid for participating in the study, regardless of condition, this does NOT count as the condition being financial. It should be something particular to the given condition."
    ),
    answer_format='{"response": "true/false/unknown", "explanation": "explanation of the answer"}',
)

In [11]:
len(prompt)

72180

In [19]:
thread = client.beta.threads.create()
thread

Thread(id='thread_LC3sKAJ39JLxlqIQADHY5vAB', created_at=1700557087, metadata={}, object='thread')

In [20]:
file = client.files.create(
    file=open(
        "/Users/amir/Documents/projects/atlas/server/files/sample_paper/A_114a_2021_WhenNudgesArent.mmd",
        "rb",
    ),
    purpose="assistants",
)

In [72]:
my_assistant = client.beta.assistants.update(
    assistant["id"],
    tools=[{"type": "retrieval"}],
    file_ids=[file.id],
)

In [9]:
def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message('asst_Q7HIiIPCC2inl0NKibrVaMYR', thread, user_input)
    return thread, run

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

def pretty_print_last(messages):
    print("# Messages")
    for i, m in enumerate(get_response(messages)):
        if i == len(list(get_response(messages))) - 1:
            print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
    return run

In [120]:
prompts = {
    "condition_name": {
        "prompt": (
            f"Give each condition a one or two-word name to describe it. "
            f"Where possible, use the label the research authors give it. Look at tables/figures to see their naming conventions for the conditions and use those if they exist. "
            f"For answer use a JSON format where you have a list of experimental conditions as follows {{'response': 'condition_name', 'explanation': 'explanation of the answer'}} and only return a valid JSON without the markdown formating the JSON."
        ),
    },
    "condition_type": {
        "prompt": (
            f"Is the condition considered a control or treatment in the original source? "
            f"If there is no control, just call all of them treatment. "
            f"If a control, specify: "
            f"do-nothing control (e.g. no email sent) "
            f"plain vanilla control (e.g., the researcher took the treatment email and stripped out all interventions from it, making it a plain control email; it is created for the research and intended to be bland) "
            f"business as usual control (e.g., the email the organization was historically sending) "
            f"business as usual do-nothing control (e.g., the organization historically did nothing) "
            f"answer should be from the following coding scheme: (Treatment/do-nothing control/plain vanilla control/business as usual control/business as usual do-nothing control) "
            f"For answer use a JSON format where you have a list of experimental conditions as follows {{'response': 'condition_type', 'explanation': 'explanation of the answer'}} and only return a valid JSON without the markdown formating the JSON."
        ),
    },
    "condition_financial": {
        "prompt": (
            f"Does the conditions you mentioned involve financial incentives? "
            f"These need not be behaviorally informed. Note this is not about the context being financial, but the condition involving financial incentives. "
            f"Note: If all participants are paid for participating in the study, regardless of condition, this does NOT count as the condition being financial. It should be something particular to the given condition. "
            f"For answer use a JSON format where you have a list of experimental conditions as follows {{'condition_name': 'the name you gave the condition', 'condition_financial': 'your answer to the question with Yes/No', 'explanation': 'explanation of the answer'}} and only return a valid JSON without the markdown formating the JSON."
        ),
    },
    "condition_educational": {
        "prompt": (
            f"Does the conditions you mentioned involve providing education or training (behaviorally informed or not)? "
            f"Note: A 'Y' here goes far beyond just providing information. It should be a more intensive educational offering. "
            f"For answer use a JSON format where you have a list of experimental conditions as follows {{'condition_name': 'the name you gave the condition', 'condition_educational': 'your answer to the question with Yes/No', 'explanation': 'explanation of the answer'}} and only return a valid JSON without the markdown formating the JSON."
        ),
    },
    "condition_forbid": {
        "prompt": (
            f"Does the conditions you mentioned forbid any options or restrict the choice situation in some way? "
            f"Meaning, there are some choices or actions the individual can no longer take if they are in the given condition. "
            f"For answer use a JSON format where you have a list of experimental conditions as follows {{'condition_name': 'the name you gave the condition', 'condition_educational': 'your answer to the question with Yes/No', 'explanation': 'explanation of the answer'}} and only return a valid JSON without the markdown formating the JSON."
        ),
    },
    "condition_channel_snailmail": {
        "prompt": (
            f"What is the channel or delivery mechanism of the condition? (It may be the same across conditions, or different.) "
            f"Is it snailmail, i.e., postal mail? This could be a letter, postcard, or other physical mail.? "
            f"For answer use a JSON format where you have a list of experimental conditions as follows [{{'condition_name':'the name you gave the condition','condition_channel_snailmail':'your answer to the question with Yes/No','explanation':'explanation of the answer'}}] and only return a valid JSON without the markdown formating the JSON."
        ),
    },
}

In [121]:
thread, run = create_thread_and_run(prompts["condition_name"]["prompt"])
run = wait_on_run(run, thread)
# pretty_print(get_response(thread))


run2 = submit_message(assistant["id"], thread, prompts["condition_financial"]["prompt"])
run2 = wait_on_run(run2, thread)
# pretty_print(get_response(thread))

run3 = submit_message(assistant["id"], thread, prompts["condition_educational"]["prompt"])
run3 = wait_on_run(run3, thread)
# pretty_print(get_response(thread))

run4 = submit_message(assistant["id"], thread, prompts["condition_forbid"]["prompt"])
run4 = wait_on_run(run4, thread)
pretty_print(get_response(thread))

# Messages
user: Give each condition a one or two-word name to describe it. Where possible, use the label the research authors give it. Look at tables/figures to see their naming conventions for the conditions and use those if they exist. For answer use a JSON format where you have a list of experimental conditions as follows {'response': 'condition_name', 'explanation': 'explanation of the answer'} and only return the JSON without the markdown formating the JSON.
assistant: {
  "response": [
    {
      "condition_name": "Baseline",
      "explanation": "The control condition without any nudges."
    },
    {
      "condition_name": "Sign",
      "explanation": "Signing-at-the-beginning nudge where customers confirmed a veracity statement before beginning the claim."
    },
    {
      "condition_name": "SN",
      "explanation": "Descriptive social norm message nudge, informing customers that others with similar profiles file claims honestly."
    },
    {
      "condition_name": "So

In [135]:
gpt_answers = []
for m in get_response(thread):
    if m.role == "assistant":
        print(json.loads(m.content[0].text.value)['response'])
        for r in json.loads(m.content[0].text.value)['response']:
            gpt_answers.append(r)

pd.DataFrame(gpt_answers)

[{'condition_name': 'Baseline', 'explanation': 'The control condition without any nudges.'}, {'condition_name': 'Sign', 'explanation': 'Signing-at-the-beginning nudge where customers confirmed a veracity statement before beginning the claim.'}, {'condition_name': 'SN', 'explanation': 'Descriptive social norm message nudge, informing customers that others with similar profiles file claims honestly.'}, {'condition_name': 'Sol', 'explanation': 'Solidarity message nudge, informing about the collective nature of insurance and encouraging considering the impact of actions on other customers.'}, {'condition_name': 'Sign × SN', 'explanation': 'Combination of signing-at-the-beginning and descriptive social norm message nudges.'}, {'condition_name': 'Sign × Sol', 'explanation': 'Combination of signing-at-the-beginning and solidarity message nudges.'}, {'condition_name': 'SN × Sol', 'explanation': 'Combination of descriptive social norm message and solidarity message nudges.'}]
[{'condition_name'

,condition_name,explanation,condition_financial,condition_educational,condition_restriction
0,Baseline,The control condition without any nudges.,NaN,NaN,NaN
1,Sign,Signing-at-the-beginning nudge where customers...,NaN,NaN,NaN
2,SN,"Descriptive social norm message nudge, informi...",NaN,NaN,NaN
3,Sol,"Solidarity message nudge, informing about the ...",NaN,NaN,NaN
4,Sign × SN,Combination of signing-at-the-beginning and de...,NaN,NaN,NaN
5,Sign × Sol,Combination of signing-at-the-beginning and so...,NaN,NaN,NaN
6,SN × Sol,Combination of descriptive social norm message...,NaN,NaN,NaN
7,Baseline,This is the control condition without any inte...,No,NaN,NaN
8,Sign,This condition involves confirmation of a vera...,No,NaN,NaN
9,SN,"Involves a descriptive social norm message, no...",No,NaN,NaN


In [124]:
condition_human[['condition_financial', 'condition_educational', 'condition_forbid']]

,condition_financial,condition_educational,condition_forbid
condition_name,,,
control,Y,N,N
four-week card offer,Y,N,N
implementation intentions (without extra lines),Y,N,N
implementation intentions (without extra linse),Y,N,N
negative social norm,Y,N,N
positive social norm,Y,N,N
social norms,Y,N,N


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

message

## GPT Turbo Chat


In [112]:
def call_gpt_turbo(article, user_prompt):
    system_prompt = "You are an assistant that reads through scientific articles and gives me the behavioral outcome or experiment conditions in the article and also a description for them."
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": article},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=4096,
        temperature=0,
    )

    return completion

Trying the same article as the langchain method

In [113]:
sample_articles = ['A_55', 'A_62', 'A_87', 'A_30']

ar_texts = {}

for ar in sample_articles:
    ar_texts[ar] = read_files_to_string(file_dict[ar])



In [114]:
behavior_human, condition_human = get_condition_behavior_human_answers(
    master_db_df, 'A_55'
)

condition_human

,condition_financial,condition_educational,condition_forbid,condition_channel_snailmail,condition_channel_email,condition_channel_paperlive,condition_channel_inperson,condition_channel_mobileapp,condition_channel_online,condition_channel_sms,condition_channel_phone,condition_channel_nothing
condition_name,,,,,,,,,,,,
BAU letter,N,N,N,Y,N,N,N,N,N,N,N,N
No letter,N,N,N,N,N,N,N,N,N,N,N,Y
Welcome letter,N,N,N,Y,N,N,N,N,N,N,N,N


In [124]:
res = call_gpt_turbo(
    ar_texts["A_55"],
    # condition_prompt["text"].format(
    #     condition="'BAU letter', 'No letter', 'Welcome letter'",
    #     question=prompts["condition_channel_snailmail"]["prompt"],
    # ),
    prompts["condition_name"]["prompt"],
)

In [125]:
res.choices[0].message.content.replace("\n", "").replace("```json", "").replace("```", "")

"{  'response': [    {      'condition_name': 'Control',      'explanation': 'Group that did not receive any communication from the ATO during the experiment.'    },    {      'condition_name': 'Treatment 1',      'explanation': 'Group that received the deterrence letter emphasizing penalties and interest charges.'    },    {      'condition_name': 'Treatment 2',      'explanation': 'Group that received the non-deterrence welcome letter emphasizing simplicity and support.'    }  ]}"

In [126]:
json.loads(
    res.choices[0]
    .message.content
    # .replace("'", '"')
    .replace("\n", "")
    .replace("```json", "")
    .replace("```", "")
)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 4 (char 3)

In [108]:
master_db_df.groupby('Paper_Exp_ID_better').nunique()

,index_condition_behavior_level,index_behavior_level,index_condition,index_behavior,index_level,measurement_timing,behavior_timing,results_sample,results_sample_pre,results_sample_post,...,Dellavigna_other_team,Szaszi_transportation_team,Szaszi_sustainability_team,Szaszi_prosocial_behavior_team,Szaszi_health_team,Szaszi_finance_team,Szaszi_education_team,Szaszi_consumer_choice_team,Szaszi_other_team,prob_of_sup
Paper_Exp_ID_better,,,,,,,,,,,,,,,,,,,,,
A_101,36,12,3,12,1,1,1,0,9,9,...,1,1,1,1,1,1,1,1,1,7
A_103,8,1,8,1,1,1,1,7,0,0,...,1,1,1,1,1,1,1,1,1,8
A_111,16,4,4,4,1,1,3,1,0,0,...,1,1,1,1,1,1,1,1,1,0
A_114,24,8,3,8,1,11,8,6,0,0,...,1,1,1,1,1,1,1,1,1,13
A_116,6,3,2,3,1,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A_93,45,9,5,9,1,4,4,7,0,0,...,1,1,1,1,1,1,1,2,1,4
A_94,6,2,3,2,1,1,1,3,3,3,...,1,1,1,1,1,1,1,1,1,3
A_95,10,1,10,1,1,1,1,10,0,0,...,1,1,1,1,1,1,1,1,1,10
